In [1]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
 87% 129M/149M [00:00<00:00, 313MB/s] 
100% 149M/149M [00:00<00:00, 310MB/s]


In [2]:
import zipfile
zip_file_path = '/content/brain-tumor-mri-dataset.zip'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('/content/brain_tumor_dataset')

In [1]:
import numpy as np
import cv2
import os
import io
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from time import time
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, LambdaCallback
from keras.layers import Input, Dropout, Dense, GlobalAveragePooling2D
from keras.models import Sequential, Model
from keras.applications.resnet import ResNet50

In [2]:
# Define labels and image size
labels = ['glioma', 'meningioma', 'notumor', 'pituitary']
image_size = 150

# Function to preprocess images
def preprocess_image(image_path):
    image = cv2.imread(image_path, 0)
    image = cv2.bilateralFilter(image, 2, 50, 50)
    image = cv2.applyColorMap(image, cv2.COLORMAP_BONE)
    image = cv2.resize(image, (image_size, image_size))
    return image

# Load images and labels
x_train = []
y_train = []
x_test = []
y_test = []

for label in labels:
    trainPath = os.path.join('/content/brain_tumor_dataset/Training', label)
    for file in tqdm(os.listdir(trainPath)):
        image_path = os.path.join(trainPath, file)
        x_train.append(preprocess_image(image_path))
        y_train.append(labels.index(label))

    testPath = os.path.join('/content/brain_tumor_dataset/Testing', label)
    for file in tqdm(os.listdir(testPath)):
        image_path = os.path.join(testPath, file)
        x_test.append(preprocess_image(image_path))
        y_test.append(labels.index(label))

# Convert lists to NumPy arrays and normalize images
x_train = np.array(x_train) / 255.0
x_test = np.array(x_test) / 255.0
y_train = np.array(y_train)
y_test = np.array(y_test)

# Shuffle training data
x_train, y_train = shuffle(x_train, y_train, random_state=42)

# Perform one-hot encoding on labels
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Split dataset into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

100%|██████████| 300/300 [00:01<00:00, 194.77it/s]


**Simple CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),  # Adjust input shape for RGB images
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(labels), activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(),  # You can adjust the learning rate if needed
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

from time import time

# Train the model
start_time = time()
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_val, y_val))
end_time = time()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
143/143 [==============================] - 9s 43ms/step - loss: 0.7382 - accuracy: 0.6973 - val_loss: 0.4519 - val_accuracy: 0.8241
Epoch 2/10
143/143 [==============================] - 4s 27ms/step - loss: 0.4582 - accuracy: 0.8350 - val_loss: 0.3371 - val_accuracy: 0.8845
Epoch 3/10
143/143 [==============================] - 4s 27ms/step - loss: 0.2907 - accuracy: 0.8919 - val_loss: 0.2896 - val_accuracy: 0.8915
Epoch 4/10
143/143 [==============================] - 4s 28ms/step - loss: 0.2223 - accuracy: 0.9208 - val_loss: 0.2542 - val_accuracy: 0.9073
Epoch 5/10
143/143 [==============================] - 4s 28ms/step - loss: 0.1678 - accuracy: 0.9416 - val_loss: 0.2206 - val_accuracy: 0.9265
Epoch 6/10
143/143 [==============================] - 4s 28ms/step - loss: 0.1296 - accuracy: 0.9501 - val_loss: 0.2284 - val_accuracy: 0.9431
Epoch 7/10
143/143 [==============================] - 4s 29ms/step - loss: 0.1120 - accuracy: 0.9582 - val_loss: 0.2141 - val_accuracy: 0.9388

In [ ]:
overfit(history)

Final Training Loss: 0.055459994822740555
Final Validation Loss: 0.21994422376155853
The model is overfitting.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

# Define the MCD model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(labels), activation='softmax')
])

# Compile the model with Monte Carlo Dropout
def mc_dropout(y_true, y_pred):
    return K.std(y_pred)

model.compile(optimizer=Adam(),  # You can adjust the learning rate if needed
              loss='categorical_crossentropy',
              metrics=['accuracy', mc_dropout])  # Add mc_dropout as a metric for evaluation


In [ ]:
from time import time

# Train the model
start_time = time()
history = model.fit(x_train, y_train,
                    batch_size=32,
                    epochs=10,
                    validation_data=(x_val, y_val))
end_time = time()


Epoch 1/10
143/143 [==============================] - 13s 44ms/step - loss: 0.7448 - accuracy: 0.7102 - mc_dropout: 0.2807 - val_loss: 0.4472 - val_accuracy: 0.8145 - val_mc_dropout: 0.3392
Epoch 2/10
143/143 [==============================] - 4s 28ms/step - loss: 0.4298 - accuracy: 0.8468 - mc_dropout: 0.3536 - val_loss: 0.3243 - val_accuracy: 0.8889 - val_mc_dropout: 0.3782
Epoch 3/10
143/143 [==============================] - 4s 26ms/step - loss: 0.2825 - accuracy: 0.8963 - mc_dropout: 0.3797 - val_loss: 0.2419 - val_accuracy: 0.9090 - val_mc_dropout: 0.3941
Epoch 4/10
143/143 [==============================] - 4s 31ms/step - loss: 0.2149 - accuracy: 0.9225 - mc_dropout: 0.3937 - val_loss: 0.2197 - val_accuracy: 0.9283 - val_mc_dropout: 0.4044
Epoch 5/10
143/143 [==============================] - 4s 28ms/step - loss: 0.1674 - accuracy: 0.9365 - mc_dropout: 0.4016 - val_loss: 0.2248 - val_accuracy: 0.9195 - val_mc_dropout: 0.4033
Epoch 6/10
143/143 [==============================] - 

ValueError: too many values to unpack (expected 2)

In [ ]:
# Evaluate the model on test data
evaluation_results = model.evaluate(x_test, y_test)
print("Evaluation Results:", evaluation_results)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {np.mean(history.history["accuracy"])}')
print(f'Validation Acc: {np.mean(history.history["val_accuracy"])}')
print(f'Test Acc: {evaluation_results[1]}')


41/41 [==============================] - 1s 15ms/step - loss: 0.2024 - accuracy: 0.9550 - mc_dropout: 0.4197
Evaluation Results: [0.2024240791797638, 0.9549961686134338, 0.41970425844192505]
Time taken: 89.93361234664917 seconds
Training Acc: 0.9143576323986053
Validation Acc: 0.9166229128837585
Test Acc: 0.9549961686134338


In [ ]:
overfit(history)


Final Training Loss: 0.0640702173113823
Final Validation Loss: 0.23365411162376404
The model is overfitting.


In [5]:
from tensorflow.keras import layers, regularizers, models, optimizers
from time import time

# Define Bayesian Neural Network model with increased capacity and regularization
def build_bcnn_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, activation="relu", kernel_regularizer=regularizers.l2(0.001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, 3, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(256, 3, activation="relu", kernel_regularizer=regularizers.l2(0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(4, activation="sigmoid")(x)  # Adjust output units to 4
    return models.Model(inputs, outputs)

# Instantiate the BCNN model
input_shape = (150, 150, 3)  # Assuming input shape is (150, 150, 3)
bcnn_model = build_bcnn_model(input_shape)

# Define and compile the Bayesian model
optimizer = optimizers.Adam(learning_rate=0.0001)  # Adjust learning rate
bcnn_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])  # Use binary crossentropy for multi-label classification

# Train the BCNN model
start_time = time()
bcnn_history = bcnn_model.fit(x_train, y_train,
                              batch_size=32,
                              epochs=10,
                              validation_data=(x_val, y_val))
end_time = time()

# Evaluate the BCNN model on test data
test_loss, test_accuracy = bcnn_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {bcnn_history.history["accuracy"][-1]}')  # Print final training accuracy
print(f'Validation Acc: {bcnn_history.history["val_accuracy"][-1]}')  # Print final validation accuracy
print(f'Test Acc: {test_accuracy}')

Epoch 1/10
143/143 [==============================] - 22s 94ms/step - loss: 0.6411 - accuracy: 0.7691 - val_loss: 1.2375 - val_accuracy: 0.2791
Epoch 2/10
143/143 [==============================] - 9s 60ms/step - loss: 0.4632 - accuracy: 0.8779 - val_loss: 1.0499 - val_accuracy: 0.3710
Epoch 3/10
143/143 [==============================] - 9s 60ms/step - loss: 0.3885 - accuracy: 0.9236 - val_loss: 0.5496 - val_accuracy: 0.7664
Epoch 4/10
143/143 [==============================] - 9s 62ms/step - loss: 0.3486 - accuracy: 0.9488 - val_loss: 0.4378 - val_accuracy: 0.8644
Epoch 5/10
143/143 [==============================] - 9s 65ms/step - loss: 0.3200 - accuracy: 0.9650 - val_loss: 0.3757 - val_accuracy: 0.9283
Epoch 6/10
143/143 [==============================] - 8s 59ms/step - loss: 0.2966 - accuracy: 0.9803 - val_loss: 0.4147 - val_accuracy: 0.8793
Epoch 7/10
143/143 [==============================] - 9s 61ms/step - loss: 0.2815 - accuracy: 0.9829 - val_loss: 0.3477 - val_accuracy: 0.933

In [5]:
def overfit(history):
  # Get the final training and validation losses
  final_train_loss = history.history['loss'][-1]
  final_val_loss = history.history['val_loss'][-1]

  # Print the final losses
  print(f'Final Training Loss: {final_train_loss}')
  print(f'Final Validation Loss: {final_val_loss}')

  # Check for overfitting
  if final_val_loss > final_train_loss:
      print('The model is overfitting.')
  else:
      print('The model is not overfitting.')

In [8]:
overfit(bcnn_history)

Final Training Loss: 0.25042837858200073
Final Validation Loss: 0.33249351382255554
The model is overfitting.


In [ ]:
from tensorflow.keras import layers, regularizers, models, optimizers
from time import time

# Define Bayesian Neural Network model with increased capacity and regularization
def build_bcnn_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, 3, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(128, 3, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Conv2D(256, 3, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.3)(x)  # Increase dropout rate
    x = layers.Dense(512, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=0.001, l2=0.001))(x)  # Add a dense layer
    x = layers.Dropout(0.6)(x)  # Add an additional dropout layer
    outputs = layers.Dense(4, activation="sigmoid")(x)  # Adjust output units to 4
    return models.Model(inputs, outputs)

# Instantiate the BCNN model
input_shape = (150, 150, 3)  # Assuming input shape is (150, 150, 3)
bcnn_model = build_bcnn_model(input_shape)

# Define and compile the Bayesian model
optimizer = optimizers.Adam(learning_rate=0.0001)  # Adjust learning rate
bcnn_model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])  # Use binary crossentropy for multi-label classification

# Train the BCNN model
start_time = time()
bcnn_history = bcnn_model.fit(x_train, y_train,
                              batch_size=32,
                              epochs=10,
                              validation_data=(x_val, y_val))
end_time = time()

# Evaluate the BCNN model on test data
test_loss, test_accuracy = bcnn_model.evaluate(x_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Print process time and metrics
print(f'Time taken: {end_time - start_time} seconds')
print(f'Training Acc: {bcnn_history.history["accuracy"][-1]}')  # Print final training accuracy
print(f'Validation Acc: {bcnn_history.history["val_accuracy"][-1]}')  # Print final validation accuracy
print(f'Test Acc: {test_accuracy}')


In [6]:
overfit(bcnn_history)

Final Training Loss: 2.8457884788513184
Final Validation Loss: 2.734614372253418
The model is not overfitting.
